<a href="https://colab.research.google.com/github/NavePnow/CS3245-Information-Retrieval-NUS/blob/master/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def build_LM(in_file):
    """
    build language models for each label
    each line in in_file contains a label and a string separated by a space
    """
    print('building language models...')
    start = time.time()
    # Language model
    dict_lm = {} 
    input_file = open(in_file, 'r')
    for line in input_file:
        [label, text] = line.split(' ', 1)
        # Generate 4-gram phrase
        strings = []
        # Q1: delete Period at end of the sentence?
        for i in range(len(text)-WINDOW_SIZE):
            strings.append(text[i: i + WINDOW_SIZE])
        # Count the phrase and feed them into language model
        for string in strings:
            if string not in dict_lm:
                # Initialize the language model
                dict_lm[string] = {}
                for tmp_label in dict_label:  
                    dict_lm[string][tmp_label] = 0
                dict_lm[string][label] = 1
            else:
                dict_lm[string][label] += 1
    input_file.close()
    # Calculate the total word count of each language
    for key, value in dict_lm.items():
        for key_l in dict_label:
            dict_label[key_l] += dict_lm[key][key_l]
    add_one = len(dict_lm)  # add-one smoothing
    dict_lm_pro = copy.deepcopy(dict_lm) # Use probability instead of count
    for key, value in dict_lm_pro.items():
        for key_l in dict_label:
            dict_lm_pro[key][key_l] = (dict_lm[key][key_l] + 1)/(add_one + dict_label[key_l]) # Formula told on lecture
    end = time.time()
    print('build language models successfully')
    print('execution time: '+ str(end-start)+ 's')
    return (dict_lm_pro)

In [0]:
def test_LM(in_file, out_file, LM):
    """
    test the language models on new strings
    each line of in_file contains a string
    you should print the most probable label for each string into out_file
    """
    print("testing language models...")
    input_file = open(in_file, 'r')
    output_file = open(out_file, 'w')
    # threshold of extraterrestrial aliens
    threshold = 0.5
    for line in input_file:
        label_pro = {'malaysian': 0, 'indonesian': 0, 'tamil': 0}
        [label, text] = line.split(' ', 1)
        strings = []
        miss_count = 0
        # Generate 4-gram phrase
        for i in range(len(text)-WINDOW_SIZE):
            strings.append(text[i: i + WINDOW_SIZE])
        # Calculate the probability
        for string in strings:
            # Ignore if phrase got absent
            if string not in LM:
                miss_count += 1
                continue
            # Use log instead of multiply
            for key in dict_label:
                label_pro[key] += math.log(LM[string][key])
        miss_pro = miss_count / len(strings)
        if miss_pro > threshold:
            result = "other"
        else:
            # Select the maximum of three languages
            result = sorted(
                label_pro.items(), key=lambda label_pro: label_pro[1], reverse=True)[0][0]
        output_file.write(result + " " + line)
    input_file.close()
    output_file.close()
    print('test language models successfully')

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import re
import nltk
import sys
import getopt
import math
import copy
import time
WINDOW_SIZE = 4
dict_label = {'malaysian': 0, 'indonesian': 0, 'tamil': 0}
# LM = build_LM(input_file_b)
# test_LM(input_file_t, output_file, LM)

LM = build_LM('/content/drive/My Drive/input.train.txt')
# test_LM(input_file_t, output_file, LM)
test_LM('/content/drive/My Drive/input.test.txt', '/content/drive/My Drive/input.predict.txt', LM)

building language models...
build language models successfully
execution time: 0.25275754928588867s
testing language models...
test language models successfully


In [6]:
!python3 /content/drive/My\ Drive/eval.py /content/drive/My\ Drive/input.predict.txt /content/drive/My\ Drive/input.correct.txt

accuracy: 20 / 20 (100.0%)


In [0]:
!python

Python 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 